In [67]:
import os
import random
import xarray as xr
import metpy
import cartopy.crs as ccrs
from pyresample import geometry, grid
import numpy as np
import matplotlib.pyplot as plt
import datetime
import s3fs
fs = s3fs.S3FileSystem(anon = True)

random.seed = 42

In [69]:
def AWSname(year, jday, utchr, minute):
    """
    appends to file for using wget -i with
    puts the links for the first file for a given band at a given hour into goescodes.txt
    band should be two digits
    all inputs are strings
    """
    urls = open("AWSurls.txt", "a")
    prefix = 'https://noaa-goes17.s3.amazonaws.com/'
    file_list = fs.ls('s3://noaa-goes17/ABI-L2-ACMC/' + year + '/' + jday + '/' + utchr)
    for file in file_list:
        if file[68:70] == minute:
            url = prefix + file[12:]
            urls.write('\n' + url)
    urls.close()

In [72]:
path = '../../GOES_Files/npy_files/'
files = []
open("AWSurls.txt", "w").close()

for i in range (20):
    file07 = ''
    while file07[19:21] != '07':
        file07 = random.choice(os.listdir(path))
    files.append(file07)
    
    year = file07[27:31]
    jday = file07[31:34]
    utchr = file07[34:36]
    minute = file07[36:38]
    
    AWSname(year, jday, utchr, minute)

In [70]:
pc = ccrs.PlateCarree()
pc_extents = [-124.25, -114.25, 32.25, 42.25]
pc_params = pc.proj4_params
target_extents = [pc_extents[0],
                    pc_extents[2],
                    pc_extents[1],
                    pc_extents[3]]
target_rows = 500
target_cols = 500
shape = [target_rows, target_cols]

In [73]:
for file in os.listdir('../../GOES_Files/nc_files/'):
    if not file.startswith('.'):
        data = xr.open_dataset('../../GOES_Files/nc_files/' + file)
        dat = data.metpy.parse_cf('BCM')
        geos = dat.metpy.cartopy_crs
        bcm = dat.data

        goes_params = geos.proj4_params

        source_area = geometry.AreaDefinition.from_cf(data)
        target_area = geometry.AreaDefinition.from_extent('CA', pc_params, shape, target_extents)

        result = grid.get_resampled_image(target_area, source_area, bcm)
        np.save('../../GOES_Files/clear_sky_mask/' + file[:-3], result)
        os.remove('../../GOES_Files/nc_files/' + file)
        data.close()

/sharedData2/nshakoor/.conda/envs/fires/lib/python3.7/site-packages/pyproj/crs/crs.py:543: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj_string = self.to_proj4()
/sharedData2/nshakoor/.conda/envs/fires/lib/python3.7/site-packages/pyproj/crs/crs.py:543: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj_string = self.to_proj4()
/sharedData2/nshakoor/.conda/envs/fires/lib/python3.7/site-packages/pyproj/crs/crs.py:543: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  p

In [ ]:
for file in files:
    for x in os.listdir('../../GOES_Files/clear_sky_mask/'):
        if x[23:34] == file[27:36]:
            mask = np.load(file)
            mask = mask.astype(bool)
            img_array = np.load(x)
            img_array[mask]=0
            #what to do with each masked file?